# 04. Model Training (Final)

> **Note:** This notebook documents the final model training that achieved A-grade metrics.
> Data paths updated to reference the `../data/` and `../models/` folders within final submission.

## Objective
Train high-performance models on the enhanced dataset to meet "Excellent (A-grade)" criteria:
- Accuracy > 70%
- ROC-AUC > 0.75
- Macro F1 > 0.70
- Improving class F1 > 0.50

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, roc_auc_score)
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported")


✅ Libraries imported


## 1. Load Enhanced Dataset


In [ ]:
data_path = '../data/trajectory_excellent.csv'
df = pd.read_csv(data_path)

print(f"Dataset Shape: {df.shape}")
print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"Institutions: {df['UNITID'].nunique()}")

# Target distribution
print("\nTarget Distribution:")
print(df['Target_Trajectory'].value_counts())

Dataset Shape: (10332, 54)
Years: 2017 - 2022
Institutions: 1722

Target Distribution:
Target_Trajectory
Stable       4791
Declining    4145
Improving    1396
Name: count, dtype: int64


## 2. Prepare Train/Test Split


In [3]:
drop_cols = ['UNITID', 'Institution_Name', 'Year', 'State', 'Target_Trajectory', 'Target_Label']
X = df.drop(columns=drop_cols)
y = df['Target_Label'].astype(int)

categorical_cols = ['Division', 'Lag1_Division']
numerical_cols = [col for col in X.columns if col not in categorical_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (8265, 48), Test shape: (2067, 48)


## 3. Define Preprocessing & Helper Functions


In [4]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numerical_cols)
    ])


def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_prob, multi_class='ovr')
    report = classification_report(y_test, y_pred, output_dict=True)
    
    metrics = {
        'Model': name,
        'Accuracy': acc,
        'ROC-AUC': roc,
        'Macro_F1': report['macro avg']['f1-score'],
        'Improving_F1': report['2']['f1-score'],
        'Report': classification_report(y_test, y_pred)
    }
    
    print("=" * 60)
    print(f"{name} Results")
    print("=" * 60)
    print(metrics['Report'])
    print(f"Accuracy: {acc:.4f} | ROC-AUC: {roc:.4f} | Macro F1: {metrics['Macro_F1']:.4f} | Improving F1: {metrics['Improving_F1']:.4f}")
    
    return metrics


## 4. Train Baseline Models


In [5]:
baseline_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression(max_iter=2000, multi_class='multinomial'))
])

baseline_pipeline.fit(X_train, y_train)
baseline_metrics = evaluate_model(baseline_pipeline, X_test, y_test, name="Logistic Regression (SMOTE)")


Logistic Regression (SMOTE) Results
              precision    recall  f1-score   support

           0       0.62      0.29      0.39       829
           1       0.55      0.86      0.67       959
           2       0.30      0.19      0.23       279

    accuracy                           0.54      2067
   macro avg       0.49      0.45      0.43      2067
weighted avg       0.54      0.54      0.50      2067

Accuracy: 0.5414 | ROC-AUC: 0.6540 | Macro F1: 0.4335 | Improving F1: 0.2343


In [6]:
rf_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(n_estimators=400, max_depth=None, min_samples_leaf=2, random_state=42, n_jobs=-1))
])

rf_pipeline.fit(X_train, y_train)
rf_metrics = evaluate_model(rf_pipeline, X_test, y_test, name="Random Forest (SMOTE)")


Random Forest (SMOTE) Results
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       829
           1       0.95      0.86      0.90       959
           2       0.59      0.67      0.63       279

    accuracy                           0.87      2067
   macro avg       0.81      0.82      0.81      2067
weighted avg       0.87      0.87      0.87      2067

Accuracy: 0.8670 | ROC-AUC: 0.9657 | Macro F1: 0.8138 | Improving F1: 0.6263


## 5. High-Performance XGBoost


In [7]:
xgb_params = {
    'n_estimators': 600,
    'max_depth': 4,
    'learning_rate': 0.03,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'min_child_weight': 2,
    'gamma': 0.1,
    'reg_lambda': 1.0,
    'reg_alpha': 0.1,
    'eval_metric': 'mlogloss',
    'random_state': 42,
    'n_jobs': -1
}

xgb_pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', XGBClassifier(**xgb_params))
])

xgb_pipeline.fit(X_train, y_train)
xgb_metrics = evaluate_model(xgb_pipeline, X_test, y_test, name="XGBoost (Enhanced)")


XGBoost (Enhanced) Results
              precision    recall  f1-score   support

           0       0.89      0.92      0.91       829
           1       0.94      0.86      0.90       959
           2       0.59      0.72      0.65       279

    accuracy                           0.86      2067
   macro avg       0.81      0.83      0.82      2067
weighted avg       0.87      0.86      0.87      2067

Accuracy: 0.8636 | ROC-AUC: 0.9648 | Macro F1: 0.8170 | Improving F1: 0.6472


## 6. Results Comparison


In [8]:
results = pd.DataFrame([
    baseline_metrics,
    rf_metrics,
    xgb_metrics
])

results[['Model', 'Accuracy', 'ROC-AUC', 'Macro_F1', 'Improving_F1']]


,Model,Accuracy,ROC-AUC,Macro_F1,Improving_F1
0,Logistic Regression (SMOTE),0.541364,0.654038,0.433503,0.234273
1,Random Forest (SMOTE),0.866957,0.965696,0.813814,0.626263
2,XGBoost (Enhanced),0.863570,0.964767,0.816952,0.647249


## 7. Save Best Model


In [ ]:
best_model = xgb_pipeline
best_model_path = '../models/trajectory_model.joblib'
joblib.dump(best_model, best_model_path)
print(f"✅ Saved excellent model to {best_model_path}")

✅ Saved excellent model to ../today/models/final_trajectory_model_excellent.joblib


## 8. Summary
- Logistic + SMOTE provides strong baseline.
- Random Forest adds nonlinear capability.
- Enhanced XGBoost meets/exceeds A-grade requirements (target >70% accuracy, >0.75 ROC-AUC, >0.70 Macro F1, >0.50 Improving F1).
- Best model saved for downstream prediction + reporting.
